<a href="https://colab.research.google.com/github/jwala1711/G-Projects/blob/main/NLP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import string
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
review1="The movie was so fantastic ! the acting and plot by hero was good"
review2="movie was so boring,I hated the acting and plot"


In [4]:
def preprocess_text(text):
  tokens=word_tokenize(text)
  tokens=[token.lower() for token in tokens]
  tokens=[token for token in tokens if token not in string.punctuation]
  tokens=[token for token in tokens if token not in stopwords.words('english')]
  tokens=[word for word in tokens if len(word)>1]
  lemmatizer=WordNetLemmatizer()
  tokens=[lemmatizer.lemmatize(token) for token in tokens]
  return ' '.join(tokens)

In [5]:
a=preprocess_text(review1)
b=preprocess_text(review2)
print(a)
print(b)

movie fantastic acting plot hero good
movie boring hated acting plot


In [6]:
vectorizer=TfidfVectorizer()
vectors=vectorizer.fit_transform([a,b])
vectors.toarray()

array([[0.33471228, 0.        , 0.47042643, 0.47042643, 0.        ,
        0.47042643, 0.33471228, 0.33471228],
       [0.37930349, 0.53309782, 0.        , 0.        , 0.53309782,
        0.        , 0.37930349, 0.37930349]])

In [7]:
cosine_sim=cosine_similarity(vectors[0],vectors[1])[0][0]
print(round(cosine_sim,3))

0.381


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/train.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [19]:
df.shape

(27481, 6)

In [23]:
df["text1_clean"]=df["text"].astype(str).apply(preprocess_text)
df["text2_clean"]=df["selected_text"].astype(str).apply(preprocess_text)


In [22]:
df.head()

,textID,text,selected_text,sentiment,text1_clean,text2_clean
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,responded going,responded going
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad miss san diego,sooo sad
2,088c60f138,my boss is bullying me...,bullying me,negative,bos bullying ...,bullying
3,9642c003ef,what interview! leave me alone,leave me alone,negative,interview leave alone,leave alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,son put release already bought,son


In [25]:
vectorizer=TfidfVectorizer()
tfidf_matrix1=vectorizer.fit_transform(df["text1_clean"])
tfidf_matrix2=vectorizer.transform(df["text2_clean"])

In [26]:
cosine_similarities=cosine_similarity(tfidf_matrix1,tfidf_matrix2)
df['cosine_similarity']=cosine_similarities.diagonal()
df.head()



,textID,text,selected_text,sentiment,text1_clean,text2_clean,cosine_similarity
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,responded going,responded going,1.000000
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad miss san diego,sooo sad,0.535921
2,088c60f138,my boss is bullying me...,bullying me,negative,bos bullying ...,bullying,0.791472
3,9642c003ef,what interview! leave me alone,leave me alone,negative,interview leave alone,leave alone,0.763914
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,son put release already bought,son,0.473126


In [44]:
def jaccab_similarity(text1,text2):
  words1=set(text1.split())
  words2=set(text2.split())
  intersection=len(words1.intersection(words2))
  union=len(words1)+len(words2)-intersection
  if union==0:
    return 0
  else:
    return  (intersection)/(union)


In [45]:
df['jaccard_similarity']=df.apply(lambda row: jaccab_similarity(row['text1_clean'],row['text2_clean']),axis=1)
df.head()

,textID,text,selected_text,sentiment,text1_clean,text2_clean,cosine_similarity,jaccard_similarity
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,responded going,responded going,1.000000,1.000000
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad miss san diego,sooo sad,0.535921,0.400000
2,088c60f138,my boss is bullying me...,bullying me,negative,bos bullying ...,bullying,0.791472,0.333333
3,9642c003ef,what interview! leave me alone,leave me alone,negative,interview leave alone,leave alone,0.763914,0.666667
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,son put release already bought,son,0.473126,0.200000
